In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt

from tqdm import tqdm
from torch.utils.data import TensorDataset, DataLoader
from functools import partial
from sklearn.decomposition import PCA

from neural_nets.MLP import *
from neural_nets.CNN import * 

from inference.bnn import *
import inference.guides as guides
import inference.likelihoods as likelihoods
import inference.priors as priors
from inference.util import *

import examples.helpers as helpers

In [ ]:
Y = torch.load('examples/data/Y.pt').float()
X = torch.load('examples/data/X.pt').float()
targets_idx = torch.load('examples/data/targets_idx.pt')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# create test set indices by randomly selecting 20% of the unique targets
test_idx = helpers.get_test_idx(targets_idx, 0.2, seed=1234)

We apply to Y a linear transformation proposed by Smithson and Verkuilen (2006), so that our response variable is bounded away from zero:
$$ \tilde Y = \frac{Y(N-1) + 1/K}{N} $$

In [ ]:
N,K = Y.shape
Y_tilde = (Y*(N-1) + 1/K)/N

In [ ]:
#split X and Y_tilde into train and test
X_train = X[~test_idx]
Y_tilde_train = Y_tilde[~test_idx]

X_test = X[test_idx]
Y_tilde_test = Y_tilde[test_idx]

## Multilayer perceptron
We use principal components of the spectra as predictors, a Dirichlet likelihood on the data, and a MLP as neural network.

In [ ]:
threshold = 0.99

pc = PCA(n_components=1000, whiten=True)
pc = pc.fit(X_train)
ncomp = np.where(np.cumsum(pc.explained_variance_ratio_)>threshold)[0][0]
print(f'{ncomp} many components explain {threshold*100}% of the variance in the training set')

In [ ]:
pc = PCA(n_components=ncomp, whiten=True)

Phi_train = torch.tensor(pc.fit_transform(X_train)).float()
Phi_test = torch.tensor(pc.transform(X_test)).float()

# create data loader
train_data = TensorDataset(Phi_train,Y_tilde_train)
test_data = TensorDataset(Phi_test,Y_tilde_test)

batchsize = 32
train_loader = DataLoader(train_data, batch_size = batchsize,
                            shuffle = True)  
test_loader = DataLoader(test_data, batch_size = batchsize,shuffle = True)

In [ ]:
wp = .01
N_train,d = Phi_train.shape

net = MLP(in_dim=d, out_dim=K, width=10, depth=2, activation="relu").to(device)
likelihood = likelihoods.Dirichlet(N_train, alternative_param=False)
prior = priors.IIDPrior((dist.Normal(torch.tensor(0.,device=device), torch.tensor(wp ** -0.5, device=device))))
bayesian_mlp = LaplaceBNN(net, prior, likelihood, approximation='subnet', S_perc=0.3, name="mlp")

In [ ]:
# MAP training
# optim = pyro.optim.ClippedAdam({"lr": 1e-1, "clip_norm": 100.0, "lrd": 0.9999})
optim = pyro.optim.ClippedAdam({"lr": 1e-2})
epochs = 200
nll_hist, nll_hist_test = bayesian_mlp.fit(train_loader, optim, epochs, num_particles=1, closed_form_kl=True, hist=True, test_loader=test_loader, test_eval_interval=1)
helpers.plot_nll(nll_hist.detach(), nll_hist_test.detach())

In [ ]:
f_predictions = bayesian_mlp.predict(Phi_test, num_predictions=100)
y_predictions = bayesian_mlp.likelihood.sample(f_predictions)

helpers.plot_predictions(y_predictions, Y_tilde_test, title='Linearized Laplace')

In [ ]:
predictive_mean = bayesian_mlp.likelihood._point_predictions(f_predictions).mean(axis=0) # rao blackwellized predictive mean
(predictive_mean.cpu() - Y_tilde_test).pow(2).mean()

In [ ]:
def predictive(input_data, num_samples=1):
    y_predictions = bayesian_mlp.likelihood.sample(bayesian_mlp.predict(input_data, num_predictions=num_samples))
    return y_predictions

cp_train, cp_test = helpers.coverage(predictive, Phi_test, Y_tilde_test, Phi_train, Y_tilde_train, M=100)

print('Train coverage by output dimension:', cp_train)
print('Test coverage by output dimension:', cp_test)

In [ ]:
bayesian_mlp.log_marginal_likelihood(train_loader)

## Convolutional Neural Network

In [ ]:
train_data = TensorDataset(X_train,Y_tilde_train)
test_data = TensorDataset(X_test,Y_tilde_test)

batchsize = 32
train_loader = DataLoader(train_data, batch_size = batchsize,
                            shuffle = True)  
batchsize_test = 32                                   
test_loader = DataLoader(test_data, batch_size=batchsize_test, shuffle = True)

In [ ]:
N_train,d = X_train.shape

ch_sizes = [9,9,1]
krnl_sizes = [41,41,41]
stride = [4,4,4]

hf_size = d
for i in range(len(ch_sizes)):
    hf_size = (hf_size - krnl_sizes[i])//stride[i] + 1
print(f"Hidden feature size: {hf_size}")
cnn = CNN(in_dim=d, out_dim=K, ch_sizes=ch_sizes, krnl_sizes=krnl_sizes, stride=stride, lin_width=hf_size, lin_depth=1,).to(device)
p = sum(p.numel() for p in cnn.parameters())
print(f"Number of parameters: {p}, Number of hidden features: {hf_size}")

In [ ]:
wp = 1.

likelihood = likelihoods.Dirichlet(N_train, alternative_param=False)
cnn_map = CNN(in_dim=d, out_dim=K, ch_sizes=ch_sizes, krnl_sizes=krnl_sizes, stride=stride, lin_width=hf_size, lin_depth=1,)
prior = priors.IIDPrior((dist.Normal(torch.tensor(0.,device=device), torch.tensor(wp ** -0.5, device=device))))
bayesian_cnn = LaplaceBNN(cnn, prior, likelihood, approximation='subnet', S_perc=0.1, name="cnn")

In [ ]:
# MAP training
# optim = pyro.optim.ClippedAdam({"lr": 1e-1, "clip_norm": 100.0, "lrd": 0.999})
optim = pyro.optim.ClippedAdam({"lr": 1e-3})
nll_hist = bayesian_cnn.fit(train_loader, optim, 500, num_particles=1, closed_form_kl=True, hist=True)
helpers.plot_nll(nll_hist)

In [ ]:
f_predictions = bayesian_cnn.predict(X_test, num_predictions=30)
y_predictions = bayesian_cnn.likelihood.sample(f_predictions)

helpers.plot_predictions(y_predictions, Y_tilde_test, title='Linearized Laplace')

In [ ]:
predictive_mean = bayesian_cnn.likelihood._point_predictions(f_predictions).mean(axis=0) # rao blackwellized predictive mean
(predictive_mean.cpu() - Y_tilde_test).pow(2).mean()

In [ ]:
def predictive(input_data, num_samples=1):
    y_predictions = bayesian_cnn.likelihood.sample(bayesian_cnn.predict(input_data, num_predictions=num_samples))
    return y_predictions

cp_train, cp_test = helpers.coverage(predictive, X_test, Y_tilde_test, X_train, Y_tilde_train, M=30)

print('Train coverage by output dimension:', cp_train)
print('Test coverage by output dimension:', cp_test)

In [ ]:
bayesian_cnn.log_marginal_likelihood(train_loader)